# Text drift detection on IMDB movie reviews

### Method

We detect drift on text data using both the [Maximum Mean Discrepancy](https://docs.seldon.io/projects/alibi-detect/en/latest/methods/mmddrift.html) and [Kolmogorov-Smirnov (K-S)](https://docs.seldon.io/projects/alibi-detect/en/latest/methods/ksdrift.html) detectors. In this example notebook we will focus on detecting covariate shift $\Delta p(x)$ as detecting predicted label distribution drift does not differ from other modalities (check [K-S](https://docs.seldon.io/projects/alibi-detect/en/latest/examples/cd_ks_cifar10.html#BBSDs) and [MMD](https://docs.seldon.io/projects/alibi-detect/en/latest/examples/cd_mmd_cifar10.html#BBSDs) drift on CIFAR-10).

It becomes however a little bit more involved when we want to pick up input data drift $\Delta p(x)$. When we deal with tabular or image data, we can either directly apply the two sample hypothesis test on the input or do the test after a preprocessing step with for instance a randomly initialized encoder as proposed in [Failing Loudly: An Empirical Study of Methods for Detecting Dataset Shift](https://arxiv.org/abs/1810.11953) (they call it an Untrained AutoEncoder or *UAE*). It is not as straightforward when dealing with text, both in string or tokenized format as they don't directly represent the semantics of the input.

As a result, we extract (contextual) embeddings for the text and detect drift on those. This procedure has a significant impact on the type of drift we detect. Strictly speaking we are not detecting $\Delta p(x)$ anymore since the whole training procedure (objective function, training data etc) for the (pre)trained embeddings has an impact on the embeddings we extract.

The library contains functionality to leverage pre-trained embeddings from [HuggingFace's transformer package](https://github.com/huggingface/transformers) but also allows you to easily use your own embeddings of choice. Both options are illustrated with examples in this notebook.

### Backend

The method works with both the *PyTorch* and *TensorFlow* frameworks.

### Dataset

Binary sentiment classification [dataset](https://ai.stanford.edu/~amaas/data/sentiment/) containing $25,000$ movie reviews for training and $25,000$ for testing.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import nlp
import numpy as np

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf
from transformers import AutoTokenizer
from alibi_detect.cd import KSDrift, MMDDrift
from alibi_detect.utils.saving import save_detector, load_detector

## Load tokenizer

In [2]:
model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /home/avl/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /home/avl/.cache/torch/transformers/5

## Load data

In [3]:
def load_dataset(dataset: str, split: str = 'test'):
    data = nlp.load_dataset(dataset)
    X, y = [], []
    for x in data[split]:
        X.append(x['text'])
        y.append(x['label'])
    X = np.array(X)
    y = np.array(y)
    return X, y

In [4]:
X, y = load_dataset('imdb', split='train')
print(X.shape, y.shape)

INFO:nlp.load:Checking /home/avl/.cache/huggingface/datasets/d3b7716978cb901261e59327d43b04c52d6d29e50eeac39bea0816865a584081.7c39fd6270c5ee55bcf2e4de23af77ef299e0df65be3f3e84454dcef7175844a.py for additional imports.
INFO:filelock:Lock 139754813800592 acquired on /home/avl/.cache/huggingface/datasets/d3b7716978cb901261e59327d43b04c52d6d29e50eeac39bea0816865a584081.7c39fd6270c5ee55bcf2e4de23af77ef299e0df65be3f3e84454dcef7175844a.py.lock
INFO:nlp.load:Found main folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/imdb/imdb.py at /home/avl/anaconda3/envs/detect/lib/python3.7/site-packages/nlp/datasets/imdb
INFO:nlp.load:Found specific version folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/imdb/imdb.py at /home/avl/anaconda3/envs/detect/lib/python3.7/site-packages/nlp/datasets/imdb/76cdbd7249ea3548c928bbf304258dab44d09cd3638d9da8d42480d1d1be3743
INFO:nlp.load:Found script file from https://s3.amazonaws.com/datasets.huggingface

(25000,) (25000,)


Let's take a look at respectively a negative and positive review:

In [5]:
labels = ['Negative', 'Positive']
print(labels[y[-1]])
print(X[-1])

Negative
This is one of the dumbest films, I've ever seen. It rips off nearly ever type of thriller and manages to make a mess of them all.<br /><br />There's not a single good line or character in the whole mess. If there was a plot, it was an afterthought and as far as acting goes, there's nothing good to say so Ill say nothing. I honestly cant understand how this type of nonsense gets produced and actually released, does somebody somewhere not at some stage think, 'Oh my god this really is a load of shite' and call it a day. Its crap like this that has people downloading illegally, the trailer looks like a completely different film, at least if you have download it, you haven't wasted your time or money Don't waste your time, this is painful.


In [6]:
print(labels[y[2]])
print(X[2])

Positive
Brilliant over-acting by Lesley Ann Warren. Best dramatic hobo lady I have ever seen, and love scenes in clothes warehouse are second to none. The corn on face is a classic, as good as anything in Blazing Saddles. The take on lawyers is also superb. After being accused of being a turncoat, selling out his boss, and being dishonest the lawyer of Pepto Bolt shrugs indifferently "I'm a lawyer" he says. Three funny words. Jeffrey Tambor, a favorite from the later Larry Sanders show, is fantastic here too as a mad millionaire who wants to crush the ghetto. His character is more malevolent than usual. The hospital scene, and the scene where the homeless invade a demolition site, are all-time classics. Look for the legs scene and the two big diggers fighting (one bleeds). This movie gets better each time I see it (which is quite often).


We split the original test set in a reference dataset and a dataset which should not be rejected under the *H0* of the statistical test. We also create imbalanced datasets and inject selected words in the reference set.

In [7]:
def random_sample(X: np.ndarray, y: np.ndarray, proba_zero: float, n: int):
    if len(y.shape) == 1:
        idx_0 = np.where(y == 0)[0]
        idx_1 = np.where(y == 1)[0]
    else:
        idx_0 = np.where(y[:, 0] == 1)[0]
        idx_1 = np.where(y[:, 1] == 1)[0]
    n_0, n_1 = int(n * proba_zero), int(n * (1 - proba_zero))
    idx_0_out = np.random.choice(idx_0, n_0, replace=False)
    idx_1_out = np.random.choice(idx_1, n_1, replace=False)
    X_out = np.concatenate([X[idx_0_out], X[idx_1_out]])
    y_out = np.concatenate([y[idx_0_out], y[idx_1_out]])
    return X_out, y_out


def padding_last(x: np.ndarray, seq_len: int) -> np.ndarray:
    try:  # try not to replace padding token
        last_token = np.where(x == 0)[0][0]
    except:  # no padding
        last_token = seq_len - 1
    return 1, last_token


def padding_first(x: np.ndarray, seq_len: int) -> np.ndarray:
    try:  # try not to replace padding token
        first_token = np.where(x == 0)[0][-1] + 2
    except:  # no padding
        first_token = 0
    return first_token, seq_len - 1


def inject_word(token: int, X: np.ndarray, perc_chg: float, padding: str = 'last'):
    seq_len = X.shape[1]
    n_chg = int(perc_chg * .01 * seq_len)
    X_cp = X.copy()
    for _ in range(X.shape[0]):
        if padding == 'last':
            first_token, last_token = padding_last(X_cp[_, :], seq_len)
        else:
            first_token, last_token = padding_first(X_cp[_, :], seq_len)
        if last_token <= n_chg:
            choice_len = seq_len
        else:
            choice_len = last_token
        idx = np.random.choice(np.arange(first_token, choice_len), n_chg, replace=False)
        X_cp[_, idx] = token
    return X_cp

Reference, *H0* and imbalanced data:

In [8]:
# frac_zero = fraction with label 0 (=negative sentiment)
n_sample = 1000
X_ref = random_sample(X, y, proba_zero=.5, n=n_sample)[0]
X_h0 = random_sample(X, y, proba_zero=.5, n=n_sample)[0]
n_imb = [.1, .9]
X_imb = {_: random_sample(X, y, proba_zero=_, n=n_sample)[0] for _ in n_imb}

Inject words in reference data:

In [9]:
words = ['fantastic', 'good', 'bad', 'horrible']
perc_chg = [1., 5.]  # % of tokens to change in an instance

words_tf = tokenizer.encode(words, return_tensors='tf')
words_tf = list(words_tf.numpy()[0, 1:-1])
max_len = 100
tokens = tokenizer.batch_encode_plus(X_ref, pad_to_max_length=True, 
                                     max_length=max_len, return_tensors='tf')
X_word = {}
for i, w in enumerate(words_tf):
    X_word[words[i]] = {}
    for p in perc_chg:
        x = inject_word(w, tokens['input_ids'].numpy(), p)
        dec = tokenizer.batch_decode(x, **dict(skip_special_tokens=True))
        X_word[words[i]][p] = np.array(dec)

## Preprocessing

First we need to specify the type of embedding we want to extract from the BERT model. We can extract embeddings from the ...

- **pooler_output**: Last layer hidden-state of the first token of the sequence (classification token; CLS) further processed by a Linear layer and a Tanh activation function. The Linear layer weights are trained from the next sentence prediction (classification) objective during pre-training. **Note**: this output is usually not a good summary of the semantic content of the input, you’re often better with averaging or pooling the sequence of hidden-states for the whole input sequence.

- **last_hidden_state**: Sequence of hidden states at the output of the last layer of the model, averaged over the tokens.

- **hidden_state**: Hidden states of the model at the output of each layer, averaged over the tokens.

- **hidden_state_cls**: See *hidden_state* but use the CLS token output.

If *hidden_state* or *hidden_state_cls* is used as embedding type, you also need to pass the layer numbers used to extract the embedding from. As an example we extract embeddings from the last 8 hidden states.

In [10]:
from alibi_detect.models.tensorflow import TransformerEmbedding

emb_type = 'hidden_state'
n_layers = 8
layers = [-_ for _ in range(1, n_layers + 1)]

embedding = TransformerEmbedding(model_name, emb_type, layers)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /home/avl/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:transformers.modeling_tf_utils:loading weights file https://cdn.huggingface.co/bert-base-cased-tf_model.h5 from cache at /home/avl/.cache/torc

Let's check what an embedding looks like:

In [11]:
tokens = tokenizer.batch_encode_plus(X[:5], pad_to_max_length=True, 
                                     max_length=max_len, return_tensors='tf')
x_emb = embedding(tokens)
print(x_emb.shape)

(5, 768)


So the BERT model's embedding space used by the drift detector consists of a $768$-dimensional vector for each instance. We will therefore first apply a dimensionality reduction step with an Untrained AutoEncoder (*UAE*) before conducting the statistical hypothesis test. We use the embedding model as the input for the UAE which then projects the embedding on a lower dimensional space.

In [12]:
tf.random.set_seed(0)

In [14]:
from alibi_detect.cd.tensorflow import UAE

enc_dim = 32
shape = (x_emb.shape[1],)

uae = UAE(input_layer=embedding, shape=shape, enc_dim=enc_dim)

Let's test this again:

In [15]:
emb_uae = uae(tokens)
print(emb_uae.shape)

(5, 32)


## K-S detector
### Initialize

We proceed to initialize the drift detector. From here on the detector works the same as for other modalities such as images. Please check the [images](https://docs.seldon.io/projects/alibi-detect/en/latest/examples/cd_ks_cifar10.html) example or the [K-S detector documentation](https://docs.seldon.io/projects/alibi-detect/en/latest/methods/ksdrift.html) for more information about each of the possible parameters.

In [16]:
from functools import partial
from alibi_detect.cd.tensorflow import preprocess_drift

# define preprocessing function
preprocess_fn = partial(preprocess_drift, model=uae, tokenizer=tokenizer, 
                        max_len=max_len, batch_size=32)

# initialize detector
cd = KSDrift(X_ref, p_val=.05, preprocess_fn=preprocess_fn)

# we can also save/load an initialised detector
filepath = 'my_path_tf'  # change to directory where detector is saved
#save_detector(cd, filepath)
#cd = load_detector(filepath)

### Detect drift

Let’s first check if drift occurs on a similar sample from the training set as the reference data.

In [17]:
preds_h0 = cd.predict(X_h0, return_p_val=True)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_h0['data']['is_drift']]))
print('p-value: {}'.format(preds_h0['data']['p_val']))

Drift? No!
p-value: [0.5360543  0.60991895 0.64755726 0.02246371 0.14833806 0.46576622
 0.60991895 0.6852314  0.6852314  0.64755726 0.08690542 0.01962691
 0.06155144 0.79439443 0.43243074 0.82795686 0.99870795 0.50035924
 0.21933001 0.3699725  0.60991895 0.96887016 0.99365413 0.28769323
 0.64755726 0.9134755  0.01121108 0.82795686 0.28769323 0.79439443
 0.7590978  0.8879386 ]


Detect drift on imbalanced and perturbed datasets:

In [18]:
for k, v in X_imb.items():
    preds = cd.predict(v, return_p_val=True)
    print('% negative sentiment {}'.format(k * 100))
    print('Drift? {}'.format(labels[preds['data']['is_drift']]))
    print('p-value: {}'.format(preds['data']['p_val']))
    print('')

% negative sentiment 10.0
Drift? Yes!
p-value: [3.69972497e-01 8.27956855e-01 9.68870163e-01 6.47557259e-01
 1.12110768e-02 1.22740539e-03 3.69972497e-01 2.13202584e-05
 3.40991944e-01 1.08282514e-01 7.22554982e-01 2.40603596e-01
 1.33834302e-01 2.87693232e-01 7.22554982e-01 8.27956855e-01
 8.27956855e-01 6.20218972e-03 7.22554982e-01 1.48338065e-01
 9.13475513e-01 2.87693232e-01 9.96931016e-01 5.72654784e-01
 3.13561678e-01 1.29345525e-02 8.87938619e-01 7.76214674e-02
 1.99518353e-01 3.77843790e-02 5.00359237e-01 9.35580969e-01]

% negative sentiment 90.0
Drift? No!
p-value: [0.12833405 0.5825486  0.32157195 0.9785308  0.07975528 0.12313652
 0.02028842 0.00162954 0.300002   0.17058046 0.63624656 0.43757161
 0.6726954  0.8787614  0.05583185 0.16792585 0.9815915  0.0267063
 0.11585598 0.18343964 0.8995234  0.11339176 0.5072882  0.1507392
 0.32593367 0.0034498  0.43747288 0.29798958 0.14292577 0.72975236
 0.27586988 0.90003824]



In [19]:
for w, probas in X_word.items():
    for p, v in probas.items():
        preds = cd.predict(v, return_p_val=True)
        print('Word: {} -- % perturbed: {}'.format(w, p))
        print('Drift? {}'.format(labels[preds['data']['is_drift']]))
        print('p-value: {}'.format(preds['data']['p_val']))
        print('')

Word: fantastic -- % perturbed: 1.0
Drift? No!
p-value: [0.9134755  0.02565915 0.2406036  0.722555   0.34099194 0.01489316
 0.05464633 0.43243074 0.21933001 0.9999727  0.8879386  0.31356168
 0.00389581 0.93558097 0.9999727  0.99870795 0.5360543  0.996931
 0.03778438 0.64755726 0.9134755  0.14833806 0.06919032 0.722555
 0.28769323 0.18111965 0.9134755  0.9134755  0.1338343  0.01121108
 0.9134755  0.7590978 ]

Word: fantastic -- % perturbed: 5.0
Drift? Yes!
p-value: [6.07078255e-04 8.23113245e-15 1.84965307e-10 6.85231388e-01
 2.43227714e-08 2.53623026e-18 7.04859247e-08 8.37208051e-03
 2.15581372e-12 1.98871276e-04 6.15514442e-02 7.89124083e-13
 0.00000000e+00 2.40344345e-03 7.22554982e-01 5.46463318e-02
 7.22554982e-01 8.69054198e-02 1.93049129e-25 1.08282514e-01
 2.13202584e-05 7.05219168e-14 1.91063212e-21 1.96269080e-02
 8.36122004e-23 2.87940366e-11 1.33834302e-01 6.47557259e-01
 3.34610052e-37 1.04535818e-26 1.11478073e-06 2.82894098e-03]

Word: good -- % perturbed: 1.0
Drift? Yes

## MMD TensorFlow detector

### Initialize

Again check the [images](https://docs.seldon.io/projects/alibi-detect/en/latest/examples/cd_mmd_cifar10.html) example or the [MMD detector documentation](https://docs.seldon.io/projects/alibi-detect/en/latest/methods/mmddrift.html) for more information about each of the possible parameters.

In [20]:
cd = MMDDrift(X_ref, p_val=.05, preprocess_fn=preprocess_fn, n_permutations=100)

### Detect drift

*H0*:

In [21]:
preds_h0 = cd.predict(X_h0)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_h0['data']['is_drift']]))
print('p-value: {}'.format(preds_h0['data']['p_val']))

Drift? No!
p-value: 0.38


Imbalanced data:

In [22]:
for k, v in X_imb.items():
    preds = cd.predict(v)
    print('% negative sentiment {}'.format(k * 100))
    print('Drift? {}'.format(labels[preds['data']['is_drift']]))
    print('p-value: {}'.format(preds['data']['p_val']))
    print('')

% negative sentiment 10.0
Drift? Yes!
p-value: 0.0

% negative sentiment 90.0
Drift? Yes!
p-value: 0.0



Perturbed data:

In [23]:
for w, probas in X_word.items():
    for p, v in probas.items():
        preds = cd.predict(v)
        print('Word: {} -- % perturbed: {}'.format(w, p))
        print('Drift? {}'.format(labels[preds['data']['is_drift']]))
        print('p-value: {}'.format(preds['data']['p_val']))
        print('')

Word: fantastic -- % perturbed: 1.0
Drift? Yes!
p-value: 0.01

Word: fantastic -- % perturbed: 5.0
Drift? Yes!
p-value: 0.0

Word: good -- % perturbed: 1.0
Drift? No!
p-value: 0.53

Word: good -- % perturbed: 5.0
Drift? Yes!
p-value: 0.0

Word: bad -- % perturbed: 1.0
Drift? No!
p-value: 0.44

Word: bad -- % perturbed: 5.0
Drift? Yes!
p-value: 0.0

Word: horrible -- % perturbed: 1.0
Drift? No!
p-value: 0.2

Word: horrible -- % perturbed: 5.0
Drift? Yes!
p-value: 0.0



## MMD PyTorch detector

## Initialize

We can run the same detector with *PyTorch* backend for both the preprocessing step and MMD implementation:

In [24]:
import torch
import torch.nn as nn

# set random seed and device
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [49]:
from alibi_detect.cd.pytorch import preprocess_drift
from alibi_detect.models.pytorch import TransformerEmbedding

embedding_pt = TransformerEmbedding(model_name, emb_type, layers)

model = nn.Sequential(
    embedding_pt,
    nn.Linear(768, 256),
    nn.ReLU(),
    nn.Linear(256, enc_dim)
).to(device).eval()

# define preprocessing function
preprocess_fn = partial(preprocess_drift, model=model, tokenizer=tokenizer, 
                        max_len=max_len, batch_size=32)

# initialise drift detector
cd = MMDDrift(X_ref, backend='pytorch', p_val=.05, 
              preprocess_fn=preprocess_fn, n_permutations=100)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /home/avl/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/bert-base-cased-pytorch_model.bin from cache at /home/avl/.cache/t

### Detect drift

*H0*:

In [50]:
preds_h0 = cd.predict(X_h0)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_h0['data']['is_drift']]))
print('p-value: {}'.format(preds_h0['data']['p_val']))

Drift? No!
p-value: 0.07999999821186066


Imbalanced data:

In [51]:
for k, v in X_imb.items():
    preds = cd.predict(v)
    print('% negative sentiment {}'.format(k * 100))
    print('Drift? {}'.format(labels[preds['data']['is_drift']]))
    print('p-value: {}'.format(preds['data']['p_val']))
    print('')

% negative sentiment 10.0
Drift? Yes!
p-value: 0.0

% negative sentiment 90.0
Drift? Yes!
p-value: 0.0



Perturbed data:

In [52]:
for w, probas in X_word.items():
    for p, v in probas.items():
        preds = cd.predict(v)
        print('Word: {} -- % perturbed: {}'.format(w, p))
        print('Drift? {}'.format(labels[preds['data']['is_drift']]))
        print('p-value: {}'.format(preds['data']['p_val']))
        print('')

Word: fantastic -- % perturbed: 1.0
Drift? Yes!
p-value: 0.0

Word: fantastic -- % perturbed: 5.0
Drift? Yes!
p-value: 0.0

Word: good -- % perturbed: 1.0
Drift? No!
p-value: 0.12999999523162842

Word: good -- % perturbed: 5.0
Drift? Yes!
p-value: 0.0

Word: bad -- % perturbed: 1.0
Drift? Yes!
p-value: 0.0

Word: bad -- % perturbed: 5.0
Drift? Yes!
p-value: 0.0

Word: horrible -- % perturbed: 1.0
Drift? Yes!
p-value: 0.019999999552965164

Word: horrible -- % perturbed: 5.0
Drift? Yes!
p-value: 0.0



## Train embeddings from scratch

So far we used pre-trained embeddings from a BERT model. We can however also use embeddings from a model trained from scratch. First we define and train a simple classification model consisting of an embedding and LSTM layer in *TensorFlow*.

### Load data and train model

In [53]:
from tensorflow.keras.datasets import imdb, reuters
from tensorflow.keras.layers import Dense, Embedding, Input, LSTM
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical

INDEX_FROM = 3
NUM_WORDS = 10000


def print_sentence(tokenized_sentence: str, id2w: dict):
    print(' '.join(id2w[_] for _ in tokenized_sentence))
    print('')
    print(tokenized_sentence)


def mapping_word_id(data):
    w2id = data.get_word_index()
    w2id = {k: (v + INDEX_FROM) for k, v in w2id.items()}
    w2id["<PAD>"] = 0
    w2id["<START>"] = 1
    w2id["<UNK>"] = 2
    w2id["<UNUSED>"] = 3
    id2w = {v: k for k, v in w2id.items()}
    return w2id, id2w


def get_dataset(dataset: str = 'imdb', max_len: int = 100):
    if dataset == 'imdb':
        data = imdb
    elif dataset == 'reuters':
        data = reuters
    else:
        raise NotImplementedError

    w2id, id2w = mapping_word_id(data)

    (X_train, y_train), (X_test, y_test) = data.load_data(
        num_words=NUM_WORDS, index_from=INDEX_FROM)
    X_train = sequence.pad_sequences(X_train, maxlen=max_len)
    X_test = sequence.pad_sequences(X_test, maxlen=max_len)
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)

    return (X_train, y_train), (X_test, y_test), (w2id, id2w)


def imdb_model(X: np.ndarray, num_words: int = 100, emb_dim: int = 128,
               lstm_dim: int = 128, output_dim: int = 2) -> tf.keras.Model:
    inputs = Input(shape=(X.shape[1:]), dtype=tf.float32)
    x = Embedding(num_words, emb_dim)(inputs)
    x = LSTM(lstm_dim, dropout=.5)(x)
    outputs = Dense(output_dim, activation=tf.nn.softmax)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

Load and tokenize data:

In [54]:
(X_train, y_train), (X_test, y_test), (word2token, token2word) = \
    get_dataset(dataset='imdb', max_len=max_len)

Let's check out an instance:

In [55]:
print_sentence(X_train[0], token2word)

cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all

[1415   33    6   22   12  215   28   77   52    5   14  407   16   82
    2    8    4  107  117 5952   15  256    4    2    7 3766    5  723
   36   71   43  530  476   26  400  317   46    7    4    2 1029   13
  104   88    4  381   15  297   98   32 2071   56   26  141    6  194
 7486   18    4  226   22   21  134  476   26  480    5  144   30 5535
   18   51   36   28  224   92   25  104    4  226   65   16   38 1334
   88   12   16  283    5   16 4472  113  103   32   15   16 5345

Define and train a simple model:

In [56]:
model = imdb_model(X=X_train, num_words=NUM_WORDS, emb_dim=256, lstm_dim=128, output_dim=2)
model.fit(X_train, y_train, batch_size=32, epochs=2, 
          shuffle=True, validation_data=(X_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 22s 885us/sample - loss: 0.4201 - accuracy: 0.8042 - val_loss: 0.3328 - val_accuracy: 0.8525
Epoch 2/2
25000/25000 [==============================] - 20s 818us/sample - loss: 0.2720 - accuracy: 0.8890 - val_loss: 0.3373 - val_accuracy: 0.8547


Extract the embedding layer from the trained model and combine with UAE preprocessing step:

In [57]:
embedding = tf.keras.Model(inputs=model.inputs, outputs=model.layers[1].output)
x_emb = embedding(X_train[:5])
print(x_emb.shape)

(5, 100, 256)


In [58]:
tf.random.set_seed(0)

shape = tuple(x_emb.shape[1:])
uae = UAE(input_layer=embedding, shape=shape, enc_dim=enc_dim)

Again, create reference, *H0* and perturbed datasets. Also test against the *Reuters* news topic classification dataset.

In [59]:
X_ref, y_ref = random_sample(X_test, y_test, proba_zero=.5, n=n_sample)
X_h0, y_h0 = random_sample(X_test, y_test, proba_zero=.5, n=n_sample)
tokens = [word2token[w] for w in words]
X_word = {}
for i, t in enumerate(tokens):
    X_word[words[i]] = {}
    for p in perc_chg:
        X_word[words[i]][p] = inject_word(t, X_ref, p, padding='first')

In [60]:
# load and tokenize Reuters dataset
(X_reut, y_reut), (w2t_reut, t2w_reut) = \
    get_dataset(dataset='reuters', max_len=max_len)[1:]

# sample random instances
idx = np.random.choice(X_reut.shape[0], n_sample, replace=False)
X_ood = X_reut[idx]

### Initialize detector and detect drift

In [61]:
from alibi_detect.cd.tensorflow import preprocess_drift

# define preprocessing function
preprocess_fn = partial(preprocess_drift, model=uae, batch_size=128)

# initialize detector
cd = KSDrift(X_ref, p_val=.05, preprocess_fn=preprocess_fn)

*H0*:

In [62]:
preds_h0 = cd.predict(X_h0)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_h0['data']['is_drift']]))
print('p-value: {}'.format(preds_h0['data']['p_val']))

Drift? No!
p-value: [0.5360543  0.18111965 0.99365413 0.10828251 0.64755726 0.9540582
 0.79439443 0.31356168 0.85929435 0.2406036  0.8879386  0.50035924
 0.50035924 0.43243074 0.7590978  0.28769323 0.31356168 0.82795686
 0.40047103 0.09710453 0.04281518 0.8879386  0.5726548  0.8879386
 0.06919032 0.722555   0.43243074 0.07762147 0.18111965 0.60991895
 0.5726548  0.40047103]


Perturbed data:

In [63]:
for w, probas in X_word.items():
    for p, v in probas.items():
        preds = cd.predict(v)
        print('Word: {} -- % perturbed: {}'.format(w, p))
        print('Drift? {}'.format(labels[preds['data']['is_drift']]))
        print('p-value: {}'.format(preds['data']['p_val']))
        print('')

Word: fantastic -- % perturbed: 1.0
Drift? No!
p-value: [0.9801618  0.96887016 0.9540582  0.82795686 0.996931   0.7590978
 0.85929435 0.96887016 0.31356168 0.8879386  0.722555   0.9134755
 0.82795686 0.93558097 0.9801618  0.6852314  0.5726548  0.9882611
 0.99365413 0.31356168 0.9540582  0.96887016 0.9998709  0.96887016
 0.82795686 0.9801618  0.82795686 0.996931   0.85929435 0.82795686
 0.40047103 0.9801618 ]

Word: fantastic -- % perturbed: 5.0
Drift? Yes!
p-value: [1.08282514e-01 7.94394433e-01 5.00359237e-01 1.08282514e-01
 3.69972497e-01 2.90806405e-04 9.69783217e-03 3.32780443e-02
 2.50427207e-10 8.87938619e-01 9.13475513e-01 7.76214674e-02
 1.96269080e-02 7.94394433e-01 3.89581337e-03 1.81119651e-01
 3.60981949e-06 5.36054313e-01 4.65766221e-01 7.13247118e-06
 7.22554982e-01 4.00471032e-01 9.35580969e-01 9.71045271e-02
 2.87693232e-01 5.00359237e-01 6.15514442e-02 4.00471032e-01
 2.40603596e-01 5.72654784e-01 2.63380647e-01 7.76214674e-02]

Word: good -- % perturbed: 1.0
Drift? No

The detector is not as sensitive as the Transformer-based K-S drift detector. The embeddings trained from scratch only trained on a small dataset and a simple model with cross-entropy loss function for 2 epochs. The pre-trained BERT model on the other hand captures semantics of the data better.

Sample from the Reuters dataset:

In [64]:
preds_ood = cd.predict(X_ood)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_ood['data']['is_drift']]))
print('p-value: {}'.format(preds_ood['data']['p_val']))

Drift? Yes!
p-value: [4.56308130e-10 8.24822544e-10 4.01514189e-05 2.43227714e-08
 5.37760343e-07 8.69054198e-02 1.48338065e-01 5.00359237e-01
 3.50604125e-04 6.15514442e-02 1.00300261e-10 1.71140861e-02
 6.15514442e-02 1.79437677e-06 6.47557259e-01 3.32311448e-03
 2.56591532e-02 1.69780876e-14 1.71140861e-02 7.22554982e-01
 2.63380647e-01 8.37208051e-03 6.14975981e-09 8.24822544e-10
 1.81119651e-01 7.21312594e-03 9.69783217e-03 2.19330013e-01
 1.18559271e-07 1.20504074e-01 6.47557259e-01 1.97830971e-07]
